In [1]:
#importing libraries
import pandas as pd
import pickle
from pandas import DataFrame, Series
import numpy as np
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt
import pybaseball as pyb
from pybaseball import statcast

In [2]:
df = pd.read_csv("/Users/steph/OneDrive/Desktop/Python/MLBRunValues.csv",low_memory=False)
df.head()
#print('Young Pyo Ko' in df['Player Name'].unique())
print(df[df['Player Name'].str.contains('Young Pyo Ko')])

          Player Name  Velocity Pitch Type  Spin Rate  Vertical Break  \
5770893  Young Pyo Ko      87.0         SI     1893.0            39.0   
5770904  Young Pyo Ko      87.5         SI     1931.0            39.0   
5770909  Young Pyo Ko      88.1         SI     2121.0            39.0   
5770923  Young Pyo Ko      87.5         SI     1971.0            44.0   
5771361  Young Pyo Ko      73.5         CH     1743.0            59.0   
...               ...       ...        ...        ...             ...   
5774208  Young Pyo Ko      74.9         CH     1941.0            59.0   
5774212  Young Pyo Ko      84.9         SI     1974.0            45.0   
5774216  Young Pyo Ko      75.4         CH     1932.0            55.0   
5774221  Young Pyo Ko      75.3         CH     2040.0            58.0   
5774225  Young Pyo Ko      86.1         SI     2019.0            41.0   

         Horizontal Break    Description  Run Values  
5770893              19.0      field_out   -0.195569  
5770904      

In [3]:
#Drop all rows with NaN values
df = df.replace([np.inf, -np.inf], np.nan)
df = df.dropna()
df = df.reset_index()
print('Young Pyo Ko' in df['Player Name'].unique())

True


In [4]:
#Create New dataframes for every seperate pitch type
dff = df[df['Pitch Type'] == 'FF']
dfc = df[df['Pitch Type'] == 'FC']
dsl = df[df['Pitch Type'] == 'SL']
dch = df[df['Pitch Type'] == 'CH']
dcb = df[df['Pitch Type'] == 'CU']
dspl = df[df['Pitch Type'] == 'FS']
dsi = df[df['Pitch Type'] == 'SI']
dsw = df[df['Pitch Type'] == 'ST']
dff.head()

,index,Player Name,Velocity,Pitch Type,Spin Rate,Vertical Break,Horizontal Break,Description,Run Values
2,2,"Romano, Jordan",99.3,FF,2344.0,16.32,-2.88,ball,0.063688
3,3,"Romano, Jordan",97.4,FF,2407.0,16.32,-4.20,foul,-0.038050
6,6,"Romano, Jordan",96.5,FF,2406.0,13.80,-1.32,ball,0.063688
7,7,"Romano, Jordan",97.9,FF,2385.0,17.04,-2.16,ball,0.063688
8,8,"Romano, Jordan",96.1,FF,2356.0,17.16,0.00,swinging_strike,-0.118125


In [47]:
#Create a new dataframe for each pitch type that lists pitcher name once the average of velocity, spin rate, vertical break, horizontal break, and release extension and the sum of run values
dff_rv = dff.groupby(['Player Name']).agg({'Velocity': 'mean', 'Spin Rate': 'mean', 'Vertical Break': 'mean','Horizontal Break': 'mean','Run Values': 'sum'})
#add a colums that is the total amount of pitches thrown
dff_rv.insert(1, 'Pitch Count', dff.groupby(['Player Name']).size())
# add a column that divides Pitch count by 100
dff_rv.insert(6, 'Pitch Count/100', dff_rv['Pitch Count']/100)
#add a column that divides run values by pitch count/30
dff_rv.insert(6, 'RV/100', dff_rv['Run Values']/dff_rv['Pitch Count/100'])
#delete rows that have less than 100 pitches thrown
dff_rv = dff_rv[dff_rv['Pitch Count'] >= 30]
#sort by run value per 100 pitches
dff_rv = dff_rv.sort_values(by=['Run Values'], ascending=True)
dff_rv = dff_rv.reset_index()
dff_rv.head()

,Player Name,Velocity,Pitch Count,Spin Rate,Vertical Break,Horizontal Break,Run Values,RV/100,Pitch Count/100
0,"Verlander, Justin",94.690244,11675,2543.342355,18.872449,-11.297196,-47.377098,-0.405800,116.75
1,"deGrom, Jacob",96.474892,8551,2370.501345,16.250015,-6.659291,-47.154813,-0.551454,85.51
2,"Scherzer, Max",94.526658,12150,2482.598107,15.072119,-11.386716,-38.014953,-0.312880,121.50
3,"Chapman, Aroldis",99.218570,5105,2469.993536,18.272110,5.338507,-32.806588,-0.642636,51.05
4,"Ryan, Joe",92.086806,2395,2194.718998,14.841720,-10.817737,-27.803867,-1.160913,23.95


In [48]:
#Repeat for Slider
dsl_rv = dsl.groupby(['Player Name']).agg({'Velocity': 'mean', 'Spin Rate': 'mean', 'Vertical Break': 'mean','Horizontal Break': 'mean', 'Run Values': 'sum'})
dsl_rv.insert(1, 'Pitch Count', dsl.groupby(['Player Name']).size())
dsl_rv.insert(6, 'Pitch Count/100', dsl_rv['Pitch Count']/100)
dsl_rv.insert(6, 'RV/100', dsl_rv['Run Values']/dsl_rv['Pitch Count/100'])
dsl_rv = dsl_rv[dsl_rv['Pitch Count'] > 10]
dsl_rv = dsl_rv.sort_values(by=['Run Values'], ascending=True)
dsl_rv = dsl_rv.reset_index()
dsl_rv.head()

,Player Name,Velocity,Pitch Count,Spin Rate,Vertical Break,Horizontal Break,Run Values,RV/100,Pitch Count/100
0,"Scherzer, Max",85.740309,4406,2335.219700,2.957295,3.189941,-94.858873,-2.152948,44.06
1,"Kershaw, Clayton",87.764891,7058,2530.812128,8.808541,-4.599564,-69.907856,-0.990477,70.58
2,"deGrom, Jacob",91.213592,4760,2485.843908,5.621950,3.361563,-46.690969,-0.980903,47.60
3,"Miller, Andrew",82.710534,3256,2546.542998,-1.372998,-11.098857,-44.088341,-1.354065,32.56
4,"Giles, Ken",86.565417,2108,2237.489089,1.186281,0.267324,-40.857246,-1.938200,21.08


In [49]:
#Repeat for Changeup
dch_rv = dch.groupby(['Player Name']).agg({'Velocity': 'mean', 'Spin Rate': 'mean', 'Vertical Break': 'mean','Horizontal Break': 'mean', 'Run Values': 'sum'})
dch_rv.insert(1, 'Pitch Count', dch.groupby(['Player Name']).size())
dch_rv.insert(6, 'Pitch Count/100', dch_rv['Pitch Count']/100)
dch_rv.insert(6, 'RV/100', dch_rv['Run Values']/dch_rv['Pitch Count/100'])
dch_rv = dch_rv[dch_rv['Pitch Count'] > 10]
dch_rv = dch_rv.sort_values(by=['Run Values'], ascending=True)
dch_rv = dch_rv.reset_index()
dch_rv.head()

,Player Name,Velocity,Pitch Count,Spin Rate,Vertical Break,Horizontal Break,Run Values,RV/100,Pitch Count/100
0,"Williams, Devin",83.832844,1702,2768.420094,-2.680188,-18.218778,-33.775072,-1.984434,17.02
1,"Iglesias, Raisel",88.129151,1578,1842.949937,4.270266,-16.116502,-28.286469,-1.792552,15.78
2,"Salazar, Danny",86.172783,1466,1418.475443,5.155744,-7.905348,-22.013449,-1.501600,14.66
3,"Sale, Chris",86.527845,3322,2056.492173,3.736580,19.479266,-21.583410,-0.649711,33.22
4,"Hamels, Cole",83.999053,2958,1668.716700,9.420081,14.043083,-21.167769,-0.715611,29.58


In [50]:
#Repeat for Curveball
dcb_rv = dcb.groupby(['Player Name']).agg({'Velocity': 'mean', 'Spin Rate': 'mean', 'Vertical Break': 'mean','Horizontal Break': 'mean', 'Run Values': 'sum'})
dcb_rv.insert(1, 'Pitch Count', dcb.groupby(['Player Name']).size())
dcb_rv.insert(6, 'Pitch Count/100', dcb_rv['Pitch Count']/100)
dcb_rv.insert(6, 'RV/100', dcb_rv['Run Values']/dcb_rv['Pitch Count/100'])
dcb_rv = dcb_rv[dcb_rv['Pitch Count'] > 10]
dcb_rv = dcb_rv.sort_values(by=['Run Values'], ascending=True)
dcb_rv = dcb_rv.reset_index()
dcb_rv.head()

,Player Name,Velocity,Pitch Count,Spin Rate,Vertical Break,Horizontal Break,Run Values,RV/100,Pitch Count/100
0,"Kluber, Corey",82.891809,4029,2548.412261,1.119702,16.522532,-63.367039,-1.572773,40.29
1,"Morton, Charlie",80.185374,6461,2976.184182,-9.999895,16.600991,-55.678112,-0.861757,64.61
2,"Strasburg, Stephen",81.719568,3102,2730.203095,-8.607582,13.091760,-31.296083,-1.008900,31.02
3,"Pressly, Ryan",82.463804,1840,3187.946196,-11.080304,15.914152,-23.189826,-1.260317,18.40
4,"Kela, Keone",83.323671,1373,2300.858704,-5.525244,2.415120,-21.229232,-1.546193,13.73


In [51]:
#Repeat for Splitter
dspl_rv = dspl.groupby(['Player Name']).agg({'Velocity': 'mean', 'Spin Rate': 'mean', 'Vertical Break': 'mean','Horizontal Break': 'mean', 'Run Values': 'sum'})
dspl_rv.insert(1, 'Pitch Count', dspl.groupby(['Player Name']).size())
dspl_rv.insert(6, 'Pitch Count/100', dspl_rv['Pitch Count']/100)
dspl_rv.insert(6, 'RV/100', dspl_rv['Run Values']/dspl_rv['Pitch Count/100'])
dspl_rv = dspl_rv[dspl_rv['Pitch Count'] > 10]
dspl_rv = dspl_rv.sort_values(by=['Run Values'], ascending=True)
dspl_rv = dspl_rv.reset_index()
dspl_rv.head()

,Player Name,Velocity,Pitch Count,Spin Rate,Vertical Break,Horizontal Break,Run Values,RV/100,Pitch Count/100
0,"Neris, Hector",86.194075,3696,1385.977002,1.119545,-10.777890,-30.946200,-0.837289,36.96
1,"Tanaka, Masahiro",87.272508,4023,1546.208054,4.479672,-11.622938,-20.497962,-0.509519,40.23
2,"Gonsolin, Tony",84.097776,1439,1573.579569,4.867880,-13.068158,-18.597948,-1.292422,14.39
3,"Gausman, Kevin",84.413321,5803,1566.532139,4.311149,-13.958070,-18.166634,-0.313056,58.03
4,"Yates, Kirby",86.647517,1168,1375.113014,1.225068,-13.660890,-12.198121,-1.044360,11.68


In [52]:
#Repeat for Sinker
dsi_rv = dsi.groupby(['Player Name']).agg({'Velocity': 'mean', 'Spin Rate': 'mean', 'Vertical Break': 'mean','Horizontal Break': 'mean', 'Run Values': 'sum'})
dsi_rv.insert(1, 'Pitch Count', dsi.groupby(['Player Name']).size())
dsi_rv.insert(6, 'Pitch Count/100', dsi_rv['Pitch Count']/100)
dsi_rv.insert(6, 'RV/100', dsi_rv['Run Values']/dsi_rv['Pitch Count/100'])
dsi_rv = dsi_rv[dsi_rv['Pitch Count'] > 10]
dsi_rv = dsi_rv.sort_values(by=['Run Values'], ascending=True)
dsi_rv = dsi_rv.reset_index()
dsi_rv.head()

,Player Name,Velocity,Pitch Count,Spin Rate,Vertical Break,Horizontal Break,Run Values,RV/100,Pitch Count/100
0,"Hader, Josh",95.556907,4655,2074.920945,17.496232,9.039287,-36.936264,-0.793475,46.55
1,"Bassitt, Chris",92.696628,4745,2121.180400,10.229159,-15.722175,-17.605475,-0.371032,47.45
2,"Alcantara, Sandy",96.844464,3342,2336.325254,7.650772,-17.295978,-16.719641,-0.500288,33.42
3,"Britton, Zack",95.479948,4194,2039.266094,6.333648,13.132303,-15.527786,-0.370238,41.94
4,"Loáisiga, Jonathan",97.941495,1458,2334.259259,11.548724,-15.927819,-15.222993,-1.044101,14.58


In [53]:
#Repeat for Sweeper
dsw_rv = dsw.groupby(['Player Name']).agg({'Velocity': 'mean', 'Spin Rate': 'mean', 'Vertical Break': 'mean','Horizontal Break': 'mean', 'Run Values': 'sum'})
dsw_rv.insert(1, 'Pitch Count', dsw.groupby(['Player Name']).size())
dsw_rv.insert(6, 'Pitch Count/100', dsw_rv['Pitch Count']/100)
dsw_rv.insert(6, 'RV/100', dsw_rv['Run Values']/dsw_rv['Pitch Count/100'])
dsw_rv = dsw_rv[dsw_rv['Pitch Count'] > 10]
dsw_rv = dsw_rv.sort_values(by=['Run Values'], ascending=True)
dsw_rv = dsw_rv.reset_index()
dsw_rv.head()

,Player Name,Velocity,Pitch Count,Spin Rate,Vertical Break,Horizontal Break,Run Values,RV/100,Pitch Count/100
0,"Ohtani, Shohei",83.864830,2087,2449.032103,3.787561,15.190206,-28.667009,-1.373599,20.87
1,"Darvish, Yu",82.307242,3176,2712.457494,-1.191045,16.459912,-27.080581,-0.852663,31.76
2,"McHugh, Collin",79.752979,2316,2792.053109,-0.361917,16.752694,-18.047778,-0.779265,23.16
3,"Ottavino, Adam",81.259796,3134,2792.358328,0.369304,17.532061,-13.516369,-0.431282,31.34
4,"Adam, Jason",80.592000,475,2712.366316,-2.082189,15.271579,-11.941999,-2.514105,4.75


In [54]:
#Repeat for Cutter
dfc_rv = dfc.groupby(['Player Name']).agg({'Velocity': 'mean', 'Spin Rate': 'mean', 'Vertical Break': 'mean','Horizontal Break': 'mean', 'Run Values': 'sum'})
dfc_rv.insert(1, 'Pitch Count', dfc.groupby(['Player Name']).size())
dfc_rv.insert(6, 'Pitch Count/100', dfc_rv['Pitch Count']/100)
dfc_rv.insert(6, 'RV/100', dfc_rv['Run Values']/dfc_rv['Pitch Count/100'])
dfc_rv = dfc_rv[dfc_rv['Pitch Count'] > 10]
dfc_rv = dfc_rv.sort_values(by=['Run Values'], ascending=True)
dfc_rv = dfc_rv.reset_index()
dfc_rv.head()

,Player Name,Velocity,Pitch Count,Spin Rate,Vertical Break,Horizontal Break,Run Values,RV/100,Pitch Count/100
0,"Jansen, Kenley",92.732588,6441,2595.206024,15.887136,6.613116,-79.945297,-1.241194,64.41
1,"Burnes, Corbin",94.833473,4045,2698.713226,11.148282,4.216761,-29.162199,-0.720944,40.45
2,"Clase, Emmanuel",99.549544,1865,2608.252547,10.558070,2.935786,-19.423863,-1.041494,18.65
3,"Colomé, Alex",89.761431,4374,2126.489712,7.078491,2.708807,-16.642824,-0.380494,43.74
4,"Trivino, Lou",92.228748,1214,2401.867381,7.569489,0.755091,-12.901133,-1.062696,12.14


In [55]:
#Download all dataframes as csv files
dff_rv.to_csv(r'/Users/steph/OneDrive/Desktop/Python/MLBRunValuesFF.csv', index = False)
dsl_rv.to_csv(r'/Users/steph/OneDrive/Desktop/Python/MLBRunValuesSL.csv', index = False)
dch_rv.to_csv(r'/Users/steph/OneDrive/Desktop/Python/MLBRunValuesCH.csv', index = False)
dcb_rv.to_csv(r'/Users/steph/OneDrive/Desktop/Python/MLBRunValuesCB.csv', index = False)
dfc_rv.to_csv(r'/Users/steph/OneDrive/Desktop/Python/MLBRunValuesFC.csv', index = False)
dspl_rv.to_csv(r'/Users/steph/OneDrive/Desktop/Python/MLBRunValuesSP.csv', index = False)
dsi_rv.to_csv(r'/Users/steph/OneDrive/Desktop/Python/MLBRunValuesSI.csv', index = False)
dsw_rv.to_csv(r'/Users/steph/OneDrive/Desktop/Python/MLBRunValuesSW.csv', index = False)

In [56]:
dspl_rv.to_csv(r'/Users/steph/OneDrive/Desktop/Python/MLBRunValuesSP.csv', index = False)

In [37]:
dcb_rv.to_csv(r'/Users/steph/OneDrive/Desktop/Python/MLBRunValuesCB.csv', index = False)